In [2]:
import pandas as pd

In [98]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

In [99]:
movie_df = pd.read_csv('movie_df.csv')
print(movie_df)

                             movie            processed_movie  \
0        Marvel Cinematic Universe  marvel cinematic universe   
1                        Star Wars                  star wars   
2         Harry Potter (franchise)               harry potter   
3             The Lion King (1994)              the lion king   
4                Shrek (Franchise)                      shrek   
...                            ...                        ...   
2776      The incredible Hulk 2007   the incredible hulk 2007   
2777                   The Killers                the killers   
2778            Logan's Run (1976)                logan's run   
2779            The 6th Day (2000)                the 6th day   
2780             The Switch (2010)                 the switch   

                   expanded-genres  \
0                              NaN   
1       Action, Adventure, Fantasy   
2                              NaN   
3      Animation, Adventure, Drama   
4     Animation, Adventure, C

In [100]:
movie_df.drop('expanded-genres', axis=1, inplace=True)


In [101]:
movie_df.to_csv('best.csv', index=False)

In [102]:
best = pd.read_csv('best.csv')

In [103]:
print(best)
print(best.dtypes)


                             movie            processed_movie  \
0        Marvel Cinematic Universe  marvel cinematic universe   
1                        Star Wars                  star wars   
2         Harry Potter (franchise)               harry potter   
3             The Lion King (1994)              the lion king   
4                Shrek (Franchise)                      shrek   
...                            ...                        ...   
2776      The incredible Hulk 2007   the incredible hulk 2007   
2777                   The Killers                the killers   
2778            Logan's Run (1976)                logan's run   
2779            The 6th Day (2000)                the 6th day   
2780             The Switch (2010)                 the switch   

                                Associated Music Genres  
0                             alternative,pop,punk,jazz  
1     acoustic,folk,punk,ambient,dubstep,hard-rock,a...  
2                             alternative,pop

In [96]:
import pandas as pd

music_genre_metrics = pd.read_csv('genre_avg_metrics.csv')

# Step 2: Calculate the mean music metric values for each genre
import pandas as pd


# Step 1: Clean up genre data for consistency
best['genres'] = best['Associated Music Genres'].apply(lambda x: [genre.strip() for genre in x.split(',')])

# Step 3: Merge the two datasets based on associated genres
merged_df = best.explode('genres').merge(music_genre_metrics, left_on='genres', right_on='track_genre')

# Step 4: Calculate the mean metrics for all genres listed in 'Associated Music Genres' and add them to each row
music_metrics = ['danceability', 'energy', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
for metric in music_metrics:
    best[metric + '_mean'] = merged_df.groupby('movie')[metric].mean()

# Display the updated dataframe
print(best)


                             movie            processed_movie  \
0        Marvel Cinematic Universe  marvel cinematic universe   
1                        Star Wars                  star wars   
2         Harry Potter (franchise)               harry potter   
3             The Lion King (1994)              the lion king   
4                Shrek (Franchise)                      shrek   
...                            ...                        ...   
2417      The incredible Hulk 2007   the incredible hulk 2007   
2418                   The Killers                the killers   
2419            Logan's Run (1976)                logan's run   
2420            The 6th Day (2000)                the 6th day   
2421             The Switch (2010)                 the switch   

                                Associated Music Genres  \
0                             alternative,pop,punk,jazz   
1     acoustic,folk,punk,ambient,dubstep,hard-rock,a...   
2                             alternative,

In [104]:
best = best.dropna(subset=['Associated Music Genres'])

# Count occurrences of each movie
nan_count = best['Associated Music Genres'].isna().sum()

print("Number of NaN values in the 'Associated Music Genres' column:", nan_count)

Number of NaN values in the 'Associated Music Genres' column: 0


In [105]:
print(movie_df)

                             movie            processed_movie  \
0        Marvel Cinematic Universe  marvel cinematic universe   
1                        Star Wars                  star wars   
2         Harry Potter (franchise)               harry potter   
3             The Lion King (1994)              the lion king   
4                Shrek (Franchise)                      shrek   
...                            ...                        ...   
2776      The incredible Hulk 2007   the incredible hulk 2007   
2777                   The Killers                the killers   
2778            Logan's Run (1976)                logan's run   
2779            The 6th Day (2000)                the 6th day   
2780             The Switch (2010)                 the switch   

                                Associated Music Genres  
0                             alternative,pop,punk,jazz  
1     acoustic,folk,punk,ambient,dubstep,hard-rock,a...  
2                             alternative,pop

In [162]:
spotify_df = pd.read_csv('millionmusic.csv')

In [107]:
spotify_df['popularity'].describe()

count    114000.000000
mean         33.238535
std          22.305078
min           0.000000
25%          17.000000
50%          35.000000
75%          50.000000
max         100.000000
Name: popularity, dtype: float64

In [108]:
spotify_df['popularity_red'] = spotify_df['popularity'].apply(lambda x: int(x/5))

In [109]:
na_count = spotify_df['track_genre'].isna().sum()

print("Number of NaN values in the 'track_genre' column:", na_count)
spotify_df.head()

Number of NaN values in the 'track_genre' column: 0


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,popularity_red
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic,14
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic,11
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic,11
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic,14
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic,16


In [110]:
#simple function to create OHE features
#this gets passed later on
def ohe_prep(df, column, new_name): 
    """ 
    Create One Hot Encoded features of a specific column

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        column (str): Column to be processed
        new_name (str): new column name to be used
        
    Returns: 
        tf_df: One hot encoded features 
    """
    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [111]:
music_genre_metrics = pd.read_csv('genre_avg_metrics.csv')

In [112]:
import pandas as pd

unique_genres = best['Associated Music Genres'].unique()

average_genre_metrics_df = pd.DataFrame(index=unique_genres, columns=music_genre_metrics.columns[1:])

for genre_combination in unique_genres:
    genres = genre_combination.split(',')
    genre_combination_metrics = []
    
    # Check if the genre combination is "Unknown"
    for genre in genres:
            # Check if the genre exists in df_music
            if genre in music_genre_metrics['track_genre'].values:
                # Append the metrics for this genre to the list
                genre_metrics = music_genre_metrics.loc[music_genre_metrics['track_genre'] == genre].iloc[:, 1:]
                genre_combination_metrics.append(genre_metrics)
    
    if genre_combination_metrics:
        combined_metrics = pd.concat(genre_combination_metrics)
        
        mean_metrics = combined_metrics.mean(axis=0)
        
        average_genre_metrics_df.loc[genre_combination] = mean_metrics

average_genre_metrics_df.reset_index(inplace=True)
average_genre_metrics_df.rename(columns={'index': 'track_genre'}, inplace=True)

df_movies = best.merge(average_genre_metrics_df, left_on='Associated Music Genres', right_on='track_genre', how='left')

print(df_movies)


                             movie            processed_movie  \
0        Marvel Cinematic Universe  marvel cinematic universe   
1                        Star Wars                  star wars   
2         Harry Potter (franchise)               harry potter   
3             The Lion King (1994)              the lion king   
4                Shrek (Franchise)                      shrek   
...                            ...                        ...   
2417      The incredible Hulk 2007   the incredible hulk 2007   
2418                   The Killers                the killers   
2419            Logan's Run (1976)                logan's run   
2420            The 6th Day (2000)                the 6th day   
2421             The Switch (2010)                 the switch   

                                Associated Music Genres  \
0                             alternative,pop,punk,jazz   
1     acoustic,folk,punk,ambient,dubstep,hard-rock,a...   
2                             alternative,

In [114]:
df_movies.to_csv('lastone.csv',index=False)

In [117]:
last = pd.read_csv('lastone.csv')

In [119]:
missing_values = spotify_df.isnull().sum()
print(missing_values)

Unnamed: 0          0
track_id            0
artists             1
album_name          1
track_name          1
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
popularity_red      0
dtype: int64


In [120]:
song_df_processed = spotify_df.dropna(subset=['artists', 'album_name', 'track_name'])

missing_values_processed = song_df_processed.isnull().sum()
print(missing_values_processed)

Unnamed: 0          0
track_id            0
artists             0
album_name          0
track_name          0
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
popularity_red      0
dtype: int64


In [139]:
last['movie'] = last['movie'].str.strip()
print(last['movie'].unique())


['Marvel Cinematic Universe' 'Star Wars' 'Harry Potter (franchise)' ...
 "Logan's Run (1976)" 'The 6th Day (2000)' 'The Switch (2010)']


In [170]:
spotify_df_filtered = pd.read_csv('spotify_filtered.csv')
spotify_df_filtered = spotify_df_filtered[~spotify_df_filtered['track_name'].str.lower().str.contains('|'.join('christmas'))]


TypeError: bad operand type for unary ~: 'float'

In [168]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Define movie and song features
movie_features = ['danceability', 'energy', 'loudness', 'mode', 'speechiness', 
                  'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
song_features = ['danceability', 'energy', 'loudness', 'mode', 'speechiness', 
                 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

def generate_recommendations_for_movie(movie_title, last, spot_df, recommended_song_ids=set(), popularity_threshold=0.3):
    spot_df_filtered = spot_df[spot_df['popularity'] > popularity_threshold]
    
    associated_genres = last[last['movie'] == movie_title]['Associated Music Genres'].iloc[0].split(',')
    
    all_recommendations = pd.DataFrame()
    
    # Iterate over each genre in the associated genres pool
    for genre in associated_genres:
        # Filter songs based on alignment with the current genre
        aligned_songs = spot_df_filtered[spot_df_filtered['track_genre'].str.contains(genre)]
        
        if not aligned_songs.empty:
            # Normalize features
            scaler = StandardScaler()
            last[movie_features] = scaler.fit_transform(last[movie_features])
            aligned_songs[song_features] = scaler.transform(aligned_songs[song_features])

            # Calculate similarity matrix
            similarity_matrix = cosine_similarity(last[movie_features], aligned_songs[song_features])

            # Exclude songs similar to those already associated with movies
            for i, row in last.iterrows():
                similar_songs_indices = similarity_matrix[i].argsort()[-20:][::-1]  # Assuming 20 is a threshold for similarity

                # Exclude similar songs from consideration
                similarity_matrix[:, similar_songs_indices] = 0

            # Rank remaining songs based on similarity scores
            song_scores = similarity_matrix.sum(axis=0)
            aligned_songs['score'] = song_scores

            # Select top 15 songs for the current genre
            recommended_songs = aligned_songs.nlargest(15, 'score')

            # Exclude recommended songs that have already been recommended before
            recommended_songs = recommended_songs[~recommended_songs.index.isin(recommended_song_ids)]

            # Update the set of recommended song IDs
            recommended_song_ids.update(recommended_songs.index)

            # Append recommended songs for the current genre to the DataFrame
            all_recommendations = pd.concat([all_recommendations, recommended_songs])
    
    # Output or further processing with all_recommendations DataFrame
    return all_recommendations, recommended_song_ids

# Example usage:
recommended_song_ids = set()
movie_title = "Marvel Cinematic Universe"
recommended_songs, recommended_song_ids = generate_recommendations_for_movie(movie_title, last, spotify_df_filtered, recommended_song_ids, popularity_threshold=0.25)
print(recommended_songs)


       Unnamed: 0                track_id  \
2000         3000  5E30LdtzQTGqRvNd7l6kG5   
2001         3001  09gysnJpfQ3ublBmJDfcEC   
2002         3002  2K7xn816oNHJZ0aVqdQsha   
2003         3003  2QjOHCTQ1Jl3zawyYOpxh6   
2004         3004  4fouWK6XVHhzl78KzQ1UjL   
2005         3005  07MDkzWARZaLEdKxo6yArG   
2006         3006  60a0Rd6pjrkxjPbaKzXjfq   
2007         3007  2nLtzopw4rPReszdYBJU6h   
2008         3008  7BoobGhD4x5K96Me0hqC8Q   
2009         3009  0eu4C55hL6x29mmeAjytzC   
2010         3010  6bTbMMEShyGyrcuIwtBlpA   
2011         3011  6rrKbzJGGDlSZgLphopS49   
2012         3012  0AUyNF6iFxMNQsNx2nhtrw   
2013         3013  6CqHve8qCxKzEd3ciOy6xj   
2014         3014  75fT19n6ZQHr67Mkh86qfD   
45000       56000  6RtnuC14DOUrvpRgS6WMQX   
45001       56001  5odlY52u43F5BjByhxg7wg   
45002       56002  0mAswKg3RCbmXAV4KYTUxM   
45003       56003  0yc6Gst2xkRu0eMLeRMGCX   
45004       56004  6CDzDgIUqeDY5g8ujExx2f   
45005       56005  322TxW77VZdX9gHynK5Xue   
45006     

In [164]:
unique_genres = spotify_df['track_genre'].unique()
print(unique_genres)

['acoustic' 'afrobeat' 'alt-rock' 'alternative' 'ambient' 'anime'
 'black-metal' 'bluegrass' 'blues' 'brazil' 'breakbeat' 'british'
 'cantopop' 'chicago-house' 'children' 'chill' 'classical' 'club' 'comedy'
 'country' 'dance' 'dancehall' 'death-metal' 'deep-house' 'detroit-techno'
 'disco' 'disney' 'drum-and-bass' 'dub' 'dubstep' 'edm' 'electro'
 'electronic' 'emo' 'folk' 'forro' 'french' 'funk' 'garage' 'german'
 'gospel' 'goth' 'grindcore' 'groove' 'grunge' 'guitar' 'happy'
 'hard-rock' 'hardcore' 'hardstyle' 'heavy-metal' 'hip-hop' 'honky-tonk'
 'house' 'idm' 'indian' 'indie-pop' 'indie' 'industrial' 'iranian'
 'j-dance' 'j-idol' 'j-pop' 'j-rock' 'jazz' 'k-pop' 'kids' 'latin'
 'latino' 'malay' 'mandopop' 'metal' 'metalcore' 'minimal-techno' 'mpb'
 'new-age' 'opera' 'pagode' 'party' 'piano' 'pop-film' 'pop' 'power-pop'
 'progressive-house' 'psych-rock' 'punk-rock' 'punk' 'r-n-b' 'reggae'
 'reggaeton' 'rock-n-roll' 'rock' 'rockabilly' 'romance' 'sad' 'salsa'
 'samba' 'sertanejo' 'show

In [135]:
# Example usage:
recommended_song_ids = set()
movie_title = "Marvel Cinematic Universe"
recommended_songs, recommended_song_ids = generate_recommendations_for_movie(movie_title, last, spotify_df, recommended_song_ids, popularity_threshold=0.25)
print(recommended_songs)

        Unnamed: 0                track_id  \
74366        74366  5jZbq4ZUGhCdKEen9hHPNc   
9089          9089  3iw959smgKf8scwjRCRjgZ   
40034        40034  3iw959smgKf8scwjRCRjgZ   
31939        31939  0zdQxu4k1jGlT2VxDokAcX   
80444        80444  2bageubuN1HjNZwTNlyh0v   
33972        33972  16w3jEiUvbHNYtEajmzh86   
106520      106520  7LQCa9eK1bc69wefMetZhz   
69742        69742  2d5lFWfRoENcOc5BTRZBgb   
36542        36542  3qGBJPzh2BpgtpvjriFxYq   
34705        34705  0GRiMv0pJBkZtB7U5QKz0N   
588            588  0c7iF5fSBYxCuwsAv2z4iI   
814            814  0NHPxGWfcBbCHBrFm3yaix   
15386        15386  7xx2Ku5iifWgkc0w2By4TR   
33209        33209  7xx2Ku5iifWgkc0w2By4TR   
11105        11105  1CRtJS94Hq3PbBZT9LuF90   

                                                  artists  \
74366                                       Alceu Valença   
9089                                        Kellen Byanca   
40034                                       Kellen Byanca   
31939              

In [166]:
cols_to_drop = ['mandopop', 'gospel', 'german', 'swedish', 'turkish', 'world-music', 'pagode', 'indian', 'acoustic', 'latino', 'honky-tonk', 'iranian', 'forro', 'sertanejo', 'j-idol', 'dancehall', 'cantopop', 'disney', 'salsa', 'malay', 'brazil', 'anime', 'samba', 'j-dance']
spotify_df_filtered = spotify_df[~spotify_df['track_genre'].isin(cols_to_drop)]
print(spotify_df_filtered)
spotify_df_filtered.to_csv('spotify_filtered.csv', index=False)

        Unnamed: 0                track_id                        artists  \
1000          1000  4pR4oQJULf7FDB54TleMyH         Limoblaze;Lecrae;Happi   
1001          1001  4Ds4bq6aqOSAp1T7DikOi4                         Criolo   
1002          1002  7nYbKsvhM88mHCmFsAIQVp                         Criolo   
1003          1003  7gVNP7rI9UBZndge0ulKfL                         Criolo   
1004          1004  1hLvWelTny8vttEEZIXVjw                         Criolo   
...            ...                     ...                            ...   
111995      111995  3TpcGANz2N705Bq4zc982H                   Roots Manuva   
111996      111996  57qWtXga1hMwSfkhLDJCKQ        Everything But The Girl   
111997      111997  0xcDUsknTawAv5VZKQ62aZ                     Wax Tailor   
111998      111998  7LVOum3l1HrbitK9AoUPd7                     Cibo Matto   
111999      111999  2Mt8qdFRaaIgzsf2bxwH4Y  Lovage;Nathaniel Merriweather   

                                               album_name  \
1000          

In [156]:
print(spotify_df['track_genre'].dtype)


TypeError: 'method' object is not subscriptable

In [157]:
print(type(spotify_df))


<class 'method'>
